In [6]:
class binary_tree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
        
    def insertLeft(self, newNode):
        if isinstance(newNode, binary_tree):
            t = newNode
        else:
            t = binary_tree(newNode)
            
        if self.leftChild is not None:
            t.leftChild = self.leftChild
        self.leftChild = t
        
    def insertRight(self, newNode):
        if isinstance(newNode, binary_tree):
            t = newNode
        else:
            t = binary_tree(newNode)
            
        if self.rightChild is not None:
            t.rightChild = self.rightChild
        self.rightChild = t
        
    def isLeaf(self):
        return not (self.leftChild or self.rightChild)
    
    def getRightChild(self):
        return self.rightChild
    
    def getLeftChild(self):
        return self.leftChild
    
    def setRootVal(self, obj):
        self.key = obj
        
    def getRootVal(self):
        return self.key
    
    def inOrder(self):
        if self.leftChild:
            self.leftChild.inOrder()
        print(self.key)
        if self.rightChild:
            self.rightChild.inOrder()
        
    def postOrder(self):
        if self.leftChild:
            self.leftChild.postOrder()
        if self.rightChild:
            self.rightChild.postOrder()
        print(self.key)
        
    def preOrder(self):
        print(self.key)
        if self.leftChild:
            self.leftChild.preOrder()
        if self.rightChild:
            self.rightChild.preOrder()
            
    def printexp(self):
        if self.leftChild:
            print('(', end = ' ')
            self.leftChild.printexp()
        print(self.key)
        if self.rightChild:
            self.rightChild.printexp()
            print(')', end = ' ')
            
    def postOrderEval(self):
        opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv}
        res1 = None
        res2 = None
        if self.leftChild:
            res1 = self.leftChild.postOrderEval()
        if self.rightChild:
            res2 = self.rightChild.postOrderEval()
        if res1 and res2:
            return opers[self.key](res1, res2)
        else:
            return self.key

test = binary_tree(1)
test.insertLeft(2)
print(test.getLeftChild().key)
test.insertRight(4)
test.insertRight(5)
print('-----')
test.inOrder()

2
-----
2
1
5
4


#### 1. Extend the buildParseTree function to handle mathematical expressions that do not have spaces between every character.


In [10]:
from pythonds.basic import Stack
from pythonds.trees import BinaryTree

def buildParseTree(fpexp):
    fplist = []
    num = ''
    i = 0
    while i < len(fpexp):
        if fpexp[i] in ['(', ')', '+', '-', '*', '/']:
            fplist.append(fpexp[i])
            i += 1
        else:
            num += fpexp[i] 
            i += 1
            while fpexp[i] not in ['(', ')', '+', '-', '*', '/']:
                num += fpexp[i]
                i += 1
            fplist.append(num)
            num = ''   
            
    print(fplist)
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree

    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()

        elif i in ['+', '-', '*', '/']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()

        elif i == ')':
            currentTree = pStack.pop()

        elif i not in ['+', '-', '*', '/', ')']:
            try:
                currentTree.setRootVal(int(i))
                parent = pStack.pop()
                currentTree = parent

            except ValueError:
                raise ValueError("token '{}' is not a valid integer".format(i))

    return eTree

pt = buildParseTree("((10+5)*3)")
pt.postorder()  #defined and explained in the next section


['(', '(', '10', '+', '5', ')', '*', '3', ')']
10
5
+
3
*


#### 2. Modify the buildParseTree and evaluate functions to handle boolean statements (and, or, and not). Remember that “not” is a unary operator, so this will complicate your code somewhat.


In [140]:
from pythonds.basic import Stack
from pythonds.trees import BinaryTree
import operator

def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree
    
    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i in ['+', '-', '*', '/', '&', '|']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        elif i == ')':
            currentTree = pStack.pop()
        elif i == '~':
            currentTree.setRootVal(i)
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i not in ['+', '-', '*', '/', '&', '|', '~', '(', ')']:
            try:
                currentTree.setRootVal(int(i))
                parent = pStack.pop()
                currentTree = parent
            except ValueError:
                raise ValueError("token '{}' is not a valid integer".format(i))
                
    return eTree

def evaluate(parseTree):
    opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv, 
             '&':operator.and_, '|':operator.or_, '~':operator.__invert__}
    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()
    if parseTree.getRootVal() == '~':
        return operator.__invert__(evaluate(leftC))
    elif leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC), evaluate(rightC))
    else:        
        return parseTree.getRootVal()


pt = buildParseTree("( ( ( 2 + 3 ) * 3 ) & ~ ( 7 | 2 ) ) ")
pt.postorder()
print(evaluate(pt))

2
3
+
3
*
7
2
|
~
&
8


#### 3. Using the findSuccessor method, write a non-recursive inorder traversal for a binary search tree.
#### 4. Modify the code for a binary search tree to make it threaded. Write a non-recursive inorder traversal method for the threaded binary search tree. A threaded binary tree maintains a reference from each node to its successor.
#### 5. Modify our implementation of the binary search tree so that it handles duplicate keys properly. That is, if a key is already in the tree then the new payload should replace the old rather than add another node with the same key.


In [124]:
class TreeNode:
    def __init__(self, key, val, left=None, right=None, parent=None, successor=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent
        self.successor = successor
        
    def hasLeftChild(self):
        return self.leftChild
    
    def hasRightChild(self):
        return self.rightChild
    
    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self
    
    def isRightChild(self):
        return self.parent and self.parent.rightChild == self
    
    def isRoot(self):
        return not self.parent
    
    def isLeaf(self):
        return not (self.rightChild or self.leftChild)
    
    def hasAnyChildren(self):
        return self.rightChild or self.leftChild
    
    def hasBothChildren(self):
        return self.rightChild and self.leftChild
    
    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                self.leftChild.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.leftChild = self.righthChild
                else:
                    self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent
        
    def findSuccessor(self): # successor is guaranteed to have no more than one child
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    succ = self.parent
                else:
                    self.parent.rightChild = None    # exclude this noode
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ
    
    def findMin(self):
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current
    
    def replaceNodeData(self, key, value, lc, rc, succ):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        self.successor = succ
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self
            
class BinarySearchTree:
    
    def __init__(self):
        self.root = None
        self.size = 0
        
    def length(self):
        return self.size
    
    def __len__(self):
        return self.size
    
    def put(self, key, val):
        if self.root:
            self._put(key, val, self.root)
        else:
            self.root = TreeNode(key, val)
        self.size = self.size + 1
        
    def _put(self, key, val, currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key, val, currentNode.leftChild)
            else:
                currentNode.leftChild = TreeNode(key, val, parent=currentNode)
        elif key > currentNode.key:
            if currentNode.hasRightChild():
                self._put(key, val, currentNode.rightChild)
            else:
                currentNode.rightChild = TreeNode(key, val, parent=currentNode)
        else:
            currentNode.replaceNodeData(key, val, currentNode.leftChild, currentNode.rightChild, currentNode.successor)
        
    def __setitem__(self, key, val):
        self.put(key, val)
        
    def get(self, key):
        if self.root:
            res = self._get(key, self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None
        
    def _get(self, key, currentNode):
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key, currentNode.leftChild)
        else:
            return self._get(key, currentNode.rightChild)
        
    def __getitem__(self, key):
        return self.get(key)
    
    def __contains__(self, key):
        if self._get(key, self.root):
            return True
        else:
            return False
        
    def delete(self, key):
        if self.size > 1:
            nodeToRemove = self._get(key, self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size - 1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')
    
    def __delitem__(self, key):
        self.delete(key)
        
    def remove(self, currentNode):
        if currentNode.isLeaf(): # leaf
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren(): # has both children
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload
        else: # has only one child
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                               currentNode.leftChild.payload,
                                               currentNode.leftChild.leftChild,
                                               currentNode.leftChild.rightChild)
            else:
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                               currentNode.rightChild.payload,
                                               currentNode.rightChild.leftChild,
                                               currentNode.rightChild.rightChild)
                    
    def inOrder(self):
        lst = []
        current = self.root.findMin()
        lst.append(current.payload)
        while current.findSuccessor():
            current = current.findSuccessor()
            lst.append(current.payload)
            
        return lst
    
    def createThread(self):
        current = self.root.findMin()
        while current.findSuccessor():
            current.successor = current.findSuccessor()
            current = current.findSuccessor()
    
    def threadInOrder(self):
        lst = []
        current = self.root.findMin()
        lst.append(current.payload)
        while current.successor:
            lst.append(current.successor.payload)
            current = current.successor
        return lst
            

if __name__ == "__main__":
    mytree = BinarySearchTree()
    mytree[3]="red"
    mytree[4]="blue"
    mytree[6]="yellow"
    mytree[2]="at"
    mytree[2]="gray"
    
    print(mytree.inOrder())
    mytree.createThread()
    print(mytree.threadInOrder())
    

['gray', 'red', 'blue', 'yellow']
['gray', 'red', 'blue', 'yellow']


#### 6. Create a binary heap with a limited heap size. In other words, the heap only keeps track of the n most important items. If the heap grows in size to more than n items the least important item is dropped.
#### 8. Using the buildHeap method, write a sorting function that can sort a list in O(nlogn) time.

In [166]:
# Assume smaller numbers are more important here. The default max size of the heap is 5. One can also customize the size when create the binary heap object.

class BinHeap:
    def __init__(self, maxLen = None):
        self.heapList = [0]
        self.currentSize = 0
        if maxLen:
            self.maxSize = maxLen
        else:
            self.maxSize = 5
        
    def percUp(self, i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                self.heapList[i], self.heapList[i // 2] = self.heapList[i // 2], self.heapList[i]
            i = i // 2
    
    def insert(self, k):
        self.heapList.append(k)
        self.currentSize += 1
        self.percUp(self.currentSize)
        if self.currentSize > self.maxSize:
            max_elmt = self.findMax()
            self.heapList.remove(max_elmt)
            self.currentSize -= 1    
            
    def findMax(self): # find max element in the highest level
        maxN = self.heapList[self.maxSize//2]
        for i in range(self.maxSize//2+1, self.maxSize+1):
            if maxN < self.heapList[i]:
                maxN = self.heapList[i]
        return maxN
        
        
    def percDown(self, i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                self.heapList[i], self.heapList[mc] = self.heapList[mc], self.heapList[i]
            else:
                break
            i = mc
            
    def minChild(self, i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i*2] < self.heapList[i*2+1]:
                return i * 2
            else:
                return i * 2 + 1
            
    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList.pop()
        self.currentSize -= 1
        self.percDown(1)
        return retval
    
    def buildHeap(self, alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while i > 0:
            self.percDown(i)
            i = i - 1
            
    def heapSort(self, alist):
        sortList = []
        l = len(alist)
        while l > 0:
            self.buildHeap(alist)
            self.heapList[1], self.heapList[-1] = self.heapList[-1], self.heapList[1]
            sortList.append(self.heapList.pop())     
            alist = self.heapList[1:]
            l -= 1
        return sortList
            
if __name__ == "__main__":
    bh = BinHeap(10)
    bh.buildHeap([9,5,6,2,3,4,10,8])
    print(bh.heapList)
    bh.insert(11)
    print(bh.heapList)
    bh.insert(15)
    print(bh.heapList)
    bh.insert(1)
    print(bh.heapList)
    bh.insert(7)
    print(bh.heapList)
    bh.delMin()
    print(bh.heapList)
    bh.delMin()
    print(bh.heapList)
    print('-----------HeapSort------------')
    print(bh.heapSort([9,5,6,2,3,4,10,8]))

[0, 2, 3, 4, 5, 9, 6, 10, 8]
[0, 2, 3, 4, 5, 9, 6, 10, 8, 11]
[0, 2, 3, 4, 5, 9, 6, 10, 8, 11, 15]
[0, 1, 2, 4, 5, 3, 6, 10, 8, 11, 9]
[0, 1, 2, 4, 5, 3, 6, 10, 8, 9, 7]
[0, 2, 3, 4, 5, 7, 6, 10, 8, 9]
[0, 3, 5, 4, 8, 7, 6, 10, 9]
-----------HeapSort------------
[2, 3, 4, 5, 6, 8, 9, 10]


#### 7. Clean up the printexp function so that it does not include an ‘extra’ set of parentheses around each number.


In [144]:
def printexp(tree):
    sVal = ""
    if tree:
        if isinstance(tree.getRootVal(), int):
            sVal = printexp(tree.getLeftChild())
            sVal = sVal + str(tree.getRootVal())
            sVal = sVal + printexp(tree.getRightChild())
        else:
            sVal = '(' + printexp(tree.getLeftChild())
            sVal = sVal + str(tree.getRootVal())
            sVal = sVal + printexp(tree.getRightChild()) + ')'
    return sVal

printexp(pt)

'(((2+3)*3)&((7|2)~))'

#### 9. Write a function that takes a parse tree for a mathematical expression and calculates the derivative of the expression with respect to some variable.


In [168]:
# skip

#### 10. Implement a binary heap as a max heap.


#### 11. Using the BinaryHeap class, implement a new class called PriorityQueue. Your PriorityQueue class should implement the constructor, plus the enqueue and dequeue methods.
